This is a simple utility notebook to update the games.csv file with the latest data from NBA.com. You want to run this notebook before backfilling the data to Hopsworks.ai for the first time.

In [1]:
import os

import pandas as pd

from datetime import datetime, timedelta
from pytz import timezone

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
)

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

In [2]:
WEBSCRAPER = 'SCRAPINGANT'  # choose between 'SCRAPINGANT' and 'SELENIUM'



**Determine How current is the existing data**

In [3]:
games_old = pd.read_csv(DATAPATH / "games.csv")

# Find the last date and season in the current dataset
last_date = games_old["GAME_DATE_EST"].max()
last_season = games_old["SEASON"].max()

# remove the time from the date
last_date = last_date.split(" ")[0]

# Determine the date of the next day to begin scraping from
start_date = datetime.strptime(last_date, "%Y-%m-%d") + timedelta(days=1)

# determine what season we are in currently
today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
if today.month >= 10:
    current_season = today.year
else:
    current_season = today.year - 1

# determine which seasons we need to scrape to catch up the data
seasons = list(range(last_season, current_season+1))


print("Last date in dataset: ", last_date)
print("Last season in dataset: ", last_season)
print("Current season: ", current_season)
print("Seasons to scrape: ", seasons)
print("Start date: ", start_date)

# if the last date in the dataset is today, then we don't need to scrape any new data
if start_date > datetime.now():
    print("No new data to scrape")
    exit()

Last date in dataset:  2023-10-27
Last season in dataset:  2023
Current season:  2023
Seasons to scrape:  [2023]
Start date:  2023-10-28 00:00:00


**Activate Webdriver**

In [4]:

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""

**Scrape New Completed Games and Format Them**

In [5]:
def update_games(driver, season, start_date, end_date)-> pd.DataFrame:

    season_types = ["Regular+Season", "PlayIn", "Playoffs"]
      
    all_season_types = pd.DataFrame()

    for season_type in season_types:
        
        df = scrape_to_dataframe(api_key=SCRAPINGANT_API_KEY, driver=driver, Season=season, DateFrom=start_date, DateTo=end_date, season_type=season_type)

        if not(df.empty):
            df = convert_columns(df)
            df = combine_home_visitor(df)
            all_season_types = pd.concat([all_season_types, df], axis=0)


    return all_season_types
    

In [6]:
new_games = pd.DataFrame()
df_season = pd.DataFrame()

for season in seasons:
    end_date = datetime.strptime(f"{season+1}-08-01", "%Y-%m-%d") # use August 1st to get all games from the current season
    print(f"Scraping season {season} from {start_date} to {end_date}")
    df_season = update_games(driver, str(season), str(start_date), str(end_date))
    new_games = pd.concat([new_games, df_season], axis=0)
    start_date = datetime.strptime(f"{season+1}-10-01", "%Y-%m-%d") # if more than 1 season, reset start date to beginning of next season


new_games

Scraping season 2023 from 2023-10-28 00:00:00 to 2024-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&Season=2023&DateFrom=2023-10-28 00:00:00&DateTo=2024-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=PlayIn&Season=2023&DateFrom=2023-10-28 00:00:00&DateTo=2024-08-01 00:00:00
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&Season=2023&DateFrom=2023-10-28 00:00:00&DateTo=2024-08-01 00:00:00


""


**Close Webdriver**

In [7]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Append to Games.csv**

In [8]:
games = pd.concat([games_old, new_games], axis=0)

games.to_csv(DATAPATH / "games.csv", index=False)

games


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27446,2023-10-25 00:00:00,22300068,NaN,1610612748,1610612765,2023,NaN,103.0,40.200,80.800,...,22.0,48.0,NaN,102.0,44.600,60.000,34.400,28.0,56.0,1
27447,2023-10-25 00:00:00,22300065,NaN,1610612752,1610612738,2023,NaN,104.0,37.100,53.800,...,24.0,47.0,NaN,108.0,48.100,84.600,30.800,18.0,46.0,0
27448,2023-10-25 00:00:00,22300072,NaN,1610612762,1610612758,2023,NaN,114.0,45.100,74.200,...,22.0,54.0,NaN,130.0,46.100,85.000,37.300,29.0,45.0,0
27449,2023-10-25 00:00:00,22300063,NaN,1610612766,1610612737,2023,NaN,116.0,50.000,73.100,...,34.0,51.0,NaN,110.0,41.900,81.800,17.200,24.0,42.0,1
